### (a) Hub Selection Constraint:

$$
\sum_{i=1}^{8} x_i = 2
$$

This constraint ensures that exactly two hubs are selected ($x_i$ is a binary variable representing whether area $i$ is selected as a hub).

---

### (b) If area $i$ is not selected as a hub, it cannot serve any other areas:

$$
\sum_{j=1}^{8} y_{ij} \leq 8 \, x_i, \quad i = 1, \dots, 8
$$

This constraint ensures that if area $i$ is not selected as a hub ($x_i = 0$), then it cannot serve any other area.

---

### (c) Each area $j$ must be served by exactly one hub, and area $i$ can serve only if it is selected as a hub:

$$
\begin{aligned}
&\sum_{i=1}^{8} y_{ij} = 1, \quad &j = 1, \dots, 8, \\
&y_{ij} \leq x_i, \quad &i,j = 1, \dots, 8.
\end{aligned}
$$

This set of constraints ensures that each area $j$ is served by exactly one hub and that area $i$ can only serve if it is selected as a hub.

---

### (d) The demand $d_j$ at area $j$ is the total traffic flux arriving at area $j$:

$$
d_j = \sum_{i=1}^{8} f_{ij}, \quad j = 1, \dots, 8.
$$

The demand $d_j$ is the total traffic flux arriving at area $j$ from all other areas.

---

### (e) The total demand $D$ is the sum of demands from all areas:

$$
D = \sum_{j=1}^{8} d_j
$$

For each hub $i$, the total demand it serves must not exceed 60% of the total demand $D$:

$$
\sum_{j=1}^{8} d_j \, y_{ij} \leq 0.6 \, D, \quad i = 1, \dots, 8.
$$

This constraint ensures load balancing for each hub, limiting the proportion of total demand that each hub can serve.

---

### (f) The transportation cost is measured by the journey time multiplied by the demand:

$$
Z_1 = \sum_{i=1}^{8} \sum_{j=1}^{8} t_{ij} \, d_j \, y_{ij}.
$$

The objective function $Z_1$ calculates the total transportation cost based on the journey times and the demands.

---

### (g) Introduce a new variable $U$ as the upper bound of all selected delivery times:

$$
\min \, Z_2 = U
$$

**Subject to:**

$$
t_{ij} \, y_{ij} \leq U, \quad i,j = 1, \dots, 8.
$$

The objective function $Z_2$ minimizes the maximum delivery time for all selected paths, ensuring that the delivery time for each path does not exceed $U$.

In [1]:
import pandas as pd, numpy as np

# 平均旅程时间表（单位：分钟）
journey_time_data = {
    'From': ['Neihu', 'Yuanshan', 'Taipei', 'Sanzhong', 'Wugu', 'Freeway Bureau', 'Linkou North', 'Linkou South'],
    'Neihu': [None, 3.19, 11.93, 13.98, 15.11, 21.35, 22.24, 30.01],
    'Yuanshan': [7.39, None, 8.26, 10.18, 11.91, 18.08, 19.12, 25.02],
    'Taipei': [9.52, 1.85, None, 1.33, 3.30, 8.04, 9.97, 13.83],
    'Sanzhong': [12.69, 4.93, 3.13, None, 2.21, 8.02, 9.24, 14.12],
    'Wugu': [17.31, 9.87, 7.46, 4.11, None, 5.10, 6.82, 11.63],
    'Freeway Bureau': [169.33, None, 177.62, None, None, None, 3.01, 8.38],
    'Linkou North': [25.80, 18.61, 16.25, 12.97, 8.30, None, None, 8.62],
    'Linkou South': [45.43, 31.94, 26.90, 22.29, 12.64, None, 6.07, None]
}

journey_time_df = pd.DataFrame(journey_time_data)
journey_time_df.set_index('From', inplace=True)

# 平均交通流量表（单位：每小时车辆数），转换为每分钟车辆数
traffic_flow_data = {
    'From': ['Neihu', 'Yuanshan', 'Taipei', 'Sanzhong', 'Wugu', 'Freeway Bureau', 'Linkou North', 'Linkou South'],
    'Neihu': [None, 131.652, 193.693, 73.508, 99.106, 0.644, 59.435, 0.016],
    'Yuanshan': [132.016, None, 310.595, 141.726, 147.136, 2.198, 84.535, 0.021],
    'Taipei': [129.421, 287.611, None, 359.530, 419.402, 6.899, 532.334, 0.128],
    'Sanzhong': [65.829, 170.747, 531.410, None, 256.016, 2.516, 218.264, 0.052],
    'Wugu': [57.799, 122.848, 467.136, 190.253, None, 2.299, 160.957, 0.054],
    'Freeway Bureau': [0.005, None, 0.003, None, None, None, 899.149, 0.277],
    'Linkou North': [37.845, 86.932, 556.076, 223.984, 151.079, None, None, 0.209],
    'Linkou South': [0.008, 0.038, 0.149, 0.082, 0.071, None, 0.234, None]
}

# 将车辆流量转换为每分钟车辆数
traffic_flow_df = pd.DataFrame(traffic_flow_data)
traffic_flow_df.set_index('From', inplace=True)
traffic_flow_df = traffic_flow_df / 60  # 转换为每分钟车辆数

regions = journey_time_df.index.tolist()
print(regions)
# 将None替换为0或者100000
for i in range(len(journey_time_df)):
    for j in range(len(journey_time_df.columns)):
        if i == j:  # 对角线上的None设为0
            if pd.isna(journey_time_df.iloc[i, j]):
                journey_time_df.iloc[i, j] = 0
        else:  # 非对角线上的None设为100000
            if pd.isna(journey_time_df.iloc[i, j]):
                journey_time_df.iloc[i, j] = 100000

# 交通流量表
for i in range(len(traffic_flow_df)):
    for j in range(len(traffic_flow_df.columns)):
        if pd.isna(traffic_flow_df.iloc[i, j]):
                traffic_flow_df.iloc[i, j] = 0

# 显示结果
jt = journey_time_df
tf = traffic_flow_df
print("\n=== Journey-time matrix after NaN→0 or 100000 ===")
print(jt)

print("\n=== Traffic-flow matrix after NaN→0 ===")
print(tf)

# 重新计算 demand
demand = tf.sum(axis=0)
total_demand = demand.sum()

print("\n=== Demand per destination (veh/min) ===")
print(demand)
print(f"\nTotal demand (veh/min): {total_demand:.4f}")

['Neihu', 'Yuanshan', 'Taipei', 'Sanzhong', 'Wugu', 'Freeway Bureau', 'Linkou North', 'Linkou South']

=== Journey-time matrix after NaN→0 or 100000 ===
                Neihu  Yuanshan  Taipei  Sanzhong   Wugu  Freeway Bureau  \
From                                                                       
Neihu            0.00      7.39    9.52     12.69  17.31          169.33   
Yuanshan         3.19      0.00    1.85      4.93   9.87       100000.00   
Taipei          11.93      8.26    0.00      3.13   7.46          177.62   
Sanzhong        13.98     10.18    1.33      0.00   4.11       100000.00   
Wugu            15.11     11.91    3.30      2.21   0.00       100000.00   
Freeway Bureau  21.35     18.08    8.04      8.02   5.10            0.00   
Linkou North    22.24     19.12    9.97      9.24   6.82            3.01   
Linkou South    30.01     25.02   13.83     14.12  11.63            8.38   

                Linkou North  Linkou South  
From                                     

In [3]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, PULP_CBC_CMD

def solve_hubs(jt_df, demand_ser, hubs_required=2, minimize_max=False):
    """Return (hubs, assignment dict, total_cost, avg_time, max_time)."""
    R = list(jt_df.index)
    prob = LpProblem("HubSelection", LpMinimize)

    # Binary decision variables
    x = {i: LpVariable(f"x_{i}", 0, 1, LpBinary) for i in R}
    y = {(i, j): LpVariable(f"y_{i}_{j}", 0, 1, LpBinary) for i in R for j in R}

    # (a) hub number
    prob += lpSum(x[i] for i in R) == hubs_required

    # (b) link y-ij to hub-i
    M = len(R)
    for i in R:
        prob += lpSum(y[i, j] for j in R) <= M * x[i]

    # (c) sole-service
    for j in R:
        prob += lpSum(y[i, j] for i in R) == 1
        for i in R:
            prob += y[i, j] <= x[i]

    # (e) load balance ≤ 60 % 总需求
    D = demand_ser.sum()
    for i in R:
        prob += lpSum(demand_ser[j] * y[i, j] for j in R) <= 0.6 * D

    # objective
    if minimize_max:                     # (i) minimax
        U = LpVariable("U", 0)
        for i in R:
            for j in R:
                prob += jt_df.loc[i, j] * y[i, j] <= U
        prob += U
    else:                                # (h) min total cost
        prob += lpSum(jt_df.loc[i, j] * demand_ser[j] * y[i, j]
                       for i in R for j in R)

    prob.solve(PULP_CBC_CMD(msg=False))

    hubs = [i for i in R if x[i].value() > 0.5]
    assign = {j: next(i for i in R if y[i, j].value() > 0.5) for j in R}

    deliv_time = {j: jt_df.loc[assign[j], j] for j in R}
    total_cost = sum(deliv_time[j] * demand_ser[j] for j in R)
    avg_time   = total_cost / D
    max_time   = max(deliv_time.values())

    return hubs, assign, total_cost, avg_time, max_time


(h)

In [4]:
hubs_Z1, assign_Z1, cost_Z1, avg_Z1, max_Z1 = solve_hubs(jt, demand, minimize_max=False)

print("===== (h) Logistics – Minimise Total Cost =====")
print("Hubs chosen:", hubs_Z1)
print(f"Total cost         : {cost_Z1:.3f}")
print(f"Average time (min) : {avg_Z1:.3f}")
print(f"Max time (min)     : {max_Z1:.3f}")

pd.DataFrame(assign_Z1.items(), columns=["Destination", "Serving Hub"])\
  .sort_values("Serving Hub")


===== (h) Logistics – Minimise Total Cost =====
Hubs chosen: ['Yuanshan', 'Linkou North']
Total cost         : 344.460
Average time (min) : 2.826
Max time (min)     : 6.820


,Destination,Serving Hub
4,Wugu,Linkou North
5,Freeway Bureau,Linkou North
6,Linkou North,Linkou North
7,Linkou South,Linkou North
0,Neihu,Yuanshan
1,Yuanshan,Yuanshan
2,Taipei,Yuanshan
3,Sanzhong,Yuanshan


(i)

In [5]:
hubs_Z2, assign_Z2, cost_Z2, avg_Z2, max_Z2 = solve_hubs(jt, demand, minimize_max=True)

print("===== (i) Publicity – Minimise Max Time =====")
print("Hubs chosen:", hubs_Z2)
print(f"Max time (min)     : {max_Z2:.3f}")
print(f"Average time (min) : {avg_Z2:.3f}")
print(f"Total cost         : {cost_Z2:.3f}")

pd.DataFrame(assign_Z2.items(), columns=["Destination", "Serving Hub"])\
  .sort_values("Serving Hub")


===== (i) Publicity – Minimise Max Time =====
Hubs chosen: ['Yuanshan', 'Linkou North']
Max time (min)     : 6.820
Average time (min) : 2.826
Total cost         : 344.460


,Destination,Serving Hub
4,Wugu,Linkou North
5,Freeway Bureau,Linkou North
6,Linkou North,Linkou North
7,Linkou South,Linkou North
0,Neihu,Yuanshan
1,Yuanshan,Yuanshan
2,Taipei,Yuanshan
3,Sanzhong,Yuanshan


(j)

In [6]:
# ---- Current single-hub scenario (Taipei) ----
current_hub = "Taipei"
cost_cur = sum(jt.loc[current_hub, j] * demand[j] for j in regions)
avg_cur  = cost_cur / total_demand
max_cur  = max(jt.loc[current_hub, j] for j in regions)

imp_cost_pct     = (cost_cur - cost_Z1) / cost_cur * 100
imp_maxtime_pct  = (max_cur  - max_Z2) / max_cur  * 100

print("===== (j) Current scenario – Taipei as sole hub =====")
print(f"Total cost         : {cost_cur:.3f}")
print(f"Average time (min) : {avg_cur:.3f}")
print(f"Max time (min)     : {max_cur:.3f}")

print("\n--- Improvement over current ---")
print(f"Cost ↓ by Logistics plan  : {imp_cost_pct:.2f} %")
print(f"Max time ↓ by Publicity plan : {imp_maxtime_pct:.2f} %")


===== (j) Current scenario – Taipei as sole hub =====
Total cost         : 3361.961
Average time (min) : 27.580
Max time (min)     : 177.620

--- Improvement over current ---
Cost ↓ by Logistics plan  : 89.75 %
Max time ↓ by Publicity plan : 96.16 %


(k) 

| Scenario | Hubs | Total cost | Avg time (min) | Max time (min) |
|----------|------|------------|----------------|----------------|
| Current  | Taipei | `cost_cur` | `avg_cur` | `max_cur` |
| Logistics (min cost) | `hubs_Z1` | `cost_Z1` | `avg_Z1` | `max_Z1` |
| Publicity (min max-time) | `hubs_Z2` | `cost_Z2` | `avg_Z2` | `max_Z2` |

- **Logistics plan** reduces total cost by **`imp_cost_pct %`**, with average time also reduced; maximum time remains acceptable at `max_Z1` min.  
- **Publicity plan** shortens the worst-case delivery by **`imp_maxtime_pct %`**, at the expense of higher cost.

